In [46]:
# import necessary packages
from lxml import etree
import requests
import lxml.html
import scispacy
import spacy
from urllib.parse import urlparse
import pandas as pd 

import scispacy
import spacy

In [ ]:
!pip install -U spacy
!pip install scispacy

In [ ]:
# install the pre-tained model
# online
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_craft_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_jnlpba_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bc5cdr_md-0.3.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_ner_bionlp13cg_md-0.3.0.tar.gz

In [37]:
# Define the pre-trained model.
nlp_craft = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_craft_md/en_ner_craft_md-0.3.0") 
nlp_jnlpha = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_jnlpba_md/en_ner_jnlpba_md-0.3.0") 
nlp_bc5cdr = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_bc5cdr_md/en_ner_bc5cdr_md-0.3.0") 
nlp_bionlp13cg = spacy.load("/Users/damon/.pyenv/versions/3.7.0/lib/python3.7/site-packages/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.3.0") 

In [38]:
# get the url and parse the link
url = "https://www.cancer.gov/about-cancer/treatment/types/targeted-therapies/targeted-therapies-fact-sheet"
rp = requests.get(url)
page = lxml.html.fromstring(rp.text)

In [39]:
# get the link of FDA approaved targeted therapies
therapies = page.xpath("//section[h2[@id = 'what-targeted-therapies-have-been-approved-for-specific-types-of-cancer']]//p//a/@href")
therapies

['/about-cancer/treatment/drugs/atezolizumab',
 '/about-cancer/treatment/drugs/nivolumab',
 '/about-cancer/treatment/drugs/durvalumab',
 '/about-cancer/treatment/drugs/avelumab',
 '/about-cancer/treatment/drugs/pembrolizumab',
 '/about-cancer/treatment/drugs/erdafitinib',
 'https://www.cancer.gov/about-cancer/treatment/drugs/enfortumabvedotin-ejfv',
 '/about-cancer/treatment/drugs/bevacizumab',
 '/about-cancer/treatment/drugs/everolimus',
 '/about-cancer/treatment/drugs/everolimus',
 '/about-cancer/treatment/drugs/tamoxifencitrate',
 '/about-cancer/treatment/drugs/toremifene',
 '/about-cancer/treatment/drugs/trastuzumab',
 '/about-cancer/treatment/drugs/fulvestrant',
 '/about-cancer/treatment/drugs/anastrozole',
 '/about-cancer/treatment/drugs/exemestane',
 '/about-cancer/treatment/drugs/lapatinibditosylate',
 '/about-cancer/treatment/drugs/letrozole',
 '/about-cancer/treatment/drugs/pertuzumab',
 '/about-cancer/treatment/drugs/ado-trastuzumab-emtansine',
 '/about-cancer/treatment/drug

In [61]:
# create empty lists to contain values
therapy_list = []
web_gov = 'https://www.cancer.gov'
geneProtein_label = []
drug_label = []
disease_label = []
dailymed_label = []

for i in therapies:
    link = ""
    
    # make sure the url is complete
    if web_gov in i:
        link = i
    else:
        link = web_gov + i
    
    # test if the url is valid
    # exist
    result = urlparse(link)
    if all([result.scheme, result.netloc, result.path]):
        therName = ""
        therBrName = ""
        
        # get the targeted therapy url and parse it
        #print(link)
        therRp = requests.get(link)
        therPage = lxml.html.fromstring(therRp.text)
        
        # extract the therapy's name
        therName = therPage.xpath("//article/div/h1/text()")
        print(therName)

        if len(therName) > 0:
            print(therName[0])
            # check duplicate therapy
            if therName[0] not in therapy_list:
                therapy_list.append(therName[0])
                
                # extract the corresponding diseases (only for bold text, but non-bold is left (could be improved))
                diseases = therPage.xpath("//article//div[h2[contains(text(),'Use in Cancer')]]/ul/li/strong")
                # remove punctuation at the end of disease
                punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
                diseaseList = []
                for disease in diseases:
                    # get the disease in string
                    dis = str(disease.text_content())
                    if dis[-1] in punc:  
                        dis = dis.replace(dis[-1], "")  
                        #print(dis)
                        # add the diseases in NCI page to the list
                        diseaseList.append(dis)
                        
                    # obtain the text associated with the disease
                    path = "//article//div[h2[contains(text(),'Use in Cancer')]]/ul/li[strong/*[contains(text(),'" + dis + "')]]"
                    diseaseContent = therPage.xpath(path)
                    #print(len(diseaseContent))
                        
                    # do NER to extract GENE and PROTEIN
                    geneProtein = ''
                    if len(diseaseContent)>0:
                        #print(diseaseContent[0].text_content())
                            
                        text = diseaseContent[0].text_content()
                        doc = nlp_craft(str(text))
                        # filter GGP (gene and gene product) type
                        for entity in doc.ents:
                            if entity.label_=='GGP':
                                geneProtein = geneProtein + ', ' + entity.text
                                print(geneProtein + ' GGP')

                        doc = nlp_jnlpha(str(text))
                        # filter PROTEIN type
                        for entity in doc.ents:
                            if entity.label_=='PROTEIN':
                                geneProtein = geneProtein + ', ' + entity.text
                                print(geneProtein + ' PROTEIN')
                        
                    # add the gene and proteins for the associated disease
                    geneProtein_label.append(geneProtein[2:])
                        
                    # add the disease and drug name in NCI to the disease/drug label list
                    disease_label.append(dis)
                    drug_label.append(therName[0]+' (general)')  
                    
                    
                
                # obtain DailyMed link
                #dailyMedURL = therPage.xpath("//article//div/p/a[text() = 'FDA label information for this drug is available at DailyMed.']/@href")
                
                #if len(dailyMedURL)>0:
                    # make sure the dailymed url valid
                 #   dailyMedURL = dailyMedURL[0]
                
                    #dailymeddrug_page = NA
                
                
    
            
    

['Atezolizumab']
Atezolizumab
, PD-L1 proteinGGP
, PD-L1 protein, PD-L1 proteinPROTEIN
, PD-L1 protein, PD-L1 protein, surgery.¹PROTEIN
, BRAF geneGGP
, BRAF gene, vemurafenibGGP
, PD-L1 proteinGGP
, PD-L1 protein, EGFR geneGGP
, PD-L1 protein, EGFR gene, ALK geneGGP
, PD-L1 protein, EGFR gene, ALK gene, EGFR geneGGP
, PD-L1 protein, EGFR gene, ALK gene, EGFR gene, ALK geneGGP
, PD-L1 protein, EGFR gene, ALK gene, EGFR gene, ALK gene, ALK geneGGP
, PD-L1 protein, EGFR gene, ALK gene, EGFR gene, ALK gene, ALK gene, PD-L1 proteinPROTEIN
, PD-L1 proteinGGP
, PD-L1 protein, PD-L1 proteinPROTEIN
['Nivolumab']
Nivolumab
, hydrochloride.¹GGP
, hydrochloride.¹, irinotecan hydrochloride.¹PROTEIN
, PD-L1 proteinGGP
, PD-L1 protein, EGFRGGP
, PD-L1 protein, EGFR, ALK geneGGP
, PD-L1 protein, EGFR, ALK gene, EGFRGGP
, PD-L1 protein, EGFR, ALK gene, EGFR, ALK geneGGP
, PD-L1 protein, EGFR, ALK gene, EGFR, ALK gene, EGFRGGP
, PD-L1 protein, EGFR, ALK gene, EGFR, ALK gene, EGFR, ALK geneGGP
, PD-L1 p

['Pembrolizumab']
Pembrolizumab
['Nivolumab']
Nivolumab
['Imatinib Mesylate']
Imatinib Mesylate
['Sunitinib Malate']
Sunitinib Malate
, imatinib mesylateGGP
['Regorafenib']
Regorafenib
['Avapritinib']
Avapritinib
, PDGFRA geneGGP
['Ripretinib']
Ripretinib
, imatinib mesylateGGP
['Denosumab']
Denosumab
, aromatasePROTEIN
, androgenGGP
['Pexidartinib Hydrochloride']
Pexidartinib Hydrochloride
['Bevacizumab']
Bevacizumab
['Sorafenib Tosylate']
Sorafenib Tosylate
['Sunitinib Malate']
Sunitinib Malate
['Pazopanib Hydrochloride']
Pazopanib Hydrochloride
['Temsirolimus']
Temsirolimus
['Everolimus']
Everolimus
['Axitinib']
Axitinib
['Nivolumab']
Nivolumab
['Cabozantinib-S-Malate']
Cabozantinib-S-Malate
, cabozantinib-s-malatePROTEIN
, cabozantinib-s-malatePROTEIN
, cabozantinib-s-malatePROTEIN
['Lenvatinib Mesylate']
Lenvatinib Mesylate
['Ipilimumab']
Ipilimumab
['Pembrolizumab']
Pembrolizumab
['Avelumab']
Avelumab
[]
['Imatinib Mesylate']
Imatinib Mesylate
['Dasatinib']
Dasatinib
, imatinib m

['Fam-Trastuzumab Deruxtecan-nxki']
Fam-Trastuzumab Deruxtecan-nxki
['Imatinib Mesylate']
Imatinib Mesylate
['Midostaurin']
Midostaurin
['Cabozantinib-S-Malate']
Cabozantinib-S-Malate
['Vandetanib']
Vandetanib
['Sorafenib Tosylate']
Sorafenib Tosylate
['Lenvatinib Mesylate']
Lenvatinib Mesylate
['Trametinib Dimethyl Sulfoxide']
Trametinib Dimethyl Sulfoxide
['Selpercatinib']
Selpercatinib
['Pralsetinib']
Pralsetinib


In [62]:
len(therapy_list), len(geneProtein_label), len(disease_label), len(drug_label)

(144, 289, 289, 289)

In [63]:
geneProtein_label

['PD-L1 protein, PD-L1 protein, surgery.¹',
 '',
 'BRAF gene, vemurafenib',
 'PD-L1 protein, EGFR gene, ALK gene, EGFR gene, ALK gene, ALK gene, PD-L1 protein',
 '',
 'PD-L1 protein, PD-L1 protein',
 '',
 'hydrochloride.¹, irinotecan hydrochloride.¹',
 '',
 '',
 '',
 'PD-L1 protein, EGFR, ALK gene, EGFR, ALK gene, EGFR, ALK gene, PD-L1 protein',
 'cabozantinib-s-malate',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'PD-L1 protein, PD-L1 protein, metastatic.¹',
 'PD-L1 protein, PD-L1 protein, chemotherapy.¹',
 '',
 '',
 '',
 'PD-L1 protein, HER2/neu, PD-L1 protein, HER2/neu',
 '',
 '',
 'orPatients',
 '',
 '',
 'EGFR gene, ALK gene, PD-L1 protein, EGFR gene, ALK gene, PD-L1 protein, EGFR, ALK gene mutations, PD-L1 protein, PD-L1 protein, EGFR',
 'therapies.¹',
 '',
 '',
 'therapies.¹',
 '',
 'PD-L1 protein, PD-L1 protein',
 'PD-L1 protein, PD-L1 protein',
 'FGFR3 gene, FGFR3 gene, FGFR2 gene',
 'PD-1, PD-L1 inhibitor, PD-1, PD-L1',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'interferon a

In [55]:
disease_label

['Breast cancer',
 'Hepatocellular carcinoma',
 'Melanoma',
 'Non-small cell lung cancer',
 'Small cell lung cancer',
 'Urothelial carcinoma',
 'Classic Hodgkin lymphoma',
 'Colorectal cancer',
 'Hepatocellular carcinoma',
 'Malignant pleural mesothelioma',
 'Melanoma',
 'Non-small cell lung cancer',
 'Renal cell carcinoma',
 'Squamous cell carcinoma of the esophagus',
 'Squamous cell carcinoma of the head and neck',
 'Urothelial carcinoma',
 'Non-small cell lung cancer',
 'Small cell lung cancer',
 'Urothelial carcinoma',
 'Merkel cell carcinoma',
 'Renal cell carcinoma',
 'Urothelial carcinoma',
 'Breast cancer',
 'Cervical cancer',
 'Classic Hodgkin lymphoma',
 'Cutaneous squamous cell carcinoma',
 'Endometrial carcinoma',
 'Gastric (stomach) cancer',
 'gastroesophageal junction cancer',
 'Hepatocellular carcinoma',
 'Melanoma',
 'Merkel cell carcinoma',
 'Microsatellite instability-high (MSI-H) or mismatch repair deficient (dMMR) cancer',
 'Non-small cell lung cancer',
 'Primary me

In [56]:
drug_label

['Atezolizumab (general)',
 'Atezolizumab (general)',
 'Atezolizumab (general)',
 'Atezolizumab (general)',
 'Atezolizumab (general)',
 'Atezolizumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Nivolumab (general)',
 'Durvalumab (general)',
 'Durvalumab (general)',
 'Durvalumab (general)',
 'Avelumab (general)',
 'Avelumab (general)',
 'Avelumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pembrolizumab (general)',
 'Pemb

In [64]:
# put the labels into a dataFrame
dict = {'geneProtein_label': geneProtein_label, 'drug_label': drug_label, 'disease_label': disease_label}  
targetTherapyDF = pd.DataFrame(dict)
targetTherapyDF[:20]

,geneProtein_label,drug_label,disease_label
0,"PD-L1 protein, PD-L1 protein, surgery.¹",Atezolizumab (general),Breast cancer
1,,Atezolizumab (general),Hepatocellular carcinoma
2,"BRAF gene, vemurafenib",Atezolizumab (general),Melanoma
3,"PD-L1 protein, EGFR gene, ALK gene, EGFR gene,...",Atezolizumab (general),Non-small cell lung cancer
4,,Atezolizumab (general),Small cell lung cancer
5,"PD-L1 protein, PD-L1 protein",Atezolizumab (general),Urothelial carcinoma
6,,Nivolumab (general),Classic Hodgkin lymphoma
7,"hydrochloride.¹, irinotecan hydrochloride.¹",Nivolumab (general),Colorectal cancer
8,,Nivolumab (general),Hepatocellular carcinoma
9,,Nivolumab (general),Malignant pleural mesothelioma


In [60]:
targetTherapyDF.geneProtein_label[7]

'hydrochloride.¹, irinotecan hydrochloride.¹'